In [1]:
import Utils
Utils.set_seed(Utils.seed)

import Dataloader,  Model
import torch
import tqdm
import numpy                as  np
import matplotlib.pyplot    as  plt
import torch.nn             as  nn


# Continuous Wavelet transform (CWT)

In [2]:
# scales = np.arange(1, Utils.Scales)

# for wavelet in reversed(Utils.wavelet_lis):
#     # Perform Continuous Wavelet Transform
#     print(wavelet)
#     coefficients, frequencies = pywt.cwt(Dataloader.X_numpy[1,:Utils.input_horizon], scales, wavelet)

#     # # Plot the results
#     # plt.figure(figsize=(10, 6))
#     # plt.imshow(np.abs(coefficients), extent=[0, 1, 1, Scales], aspect='auto', cmap='jet')
#     # # plt.colorbar(label='Magnitude')
#     # plt.title('Continuous Wavelet Transform')
#     # plt.xlabel('Time')
#     # plt.ylabel('Scale')
#     # plt.show()

# if Utils.Coefficient_Real:
#     coefficients = np.abs(coefficients)

In [3]:
prediction_input_size   = Utils.input_horizon
epochs                  = Utils.num_epochs

model     = Model.Structure().to(device = Utils.Device)
criterion = torch.nn.BCELoss()#torch.nn.MSELoss()
optimizer = torch.optim.SGD(
                            model.parameters(),
                            lr=1e-6,
                            # momentum=0.9,
                            # weight_decay=1e-4
                            )

In [ ]:
loss_t = []

ll = (Dataloader.X_numpy.shape[1]//prediction_input_size)-1
for epoch in range(1,epochs+3):
    print(ll)

    for test in range(Dataloader.X_numpy.shape[0]):
        loop_train = tqdm.tqdm(range(ll),total=ll,desc="train",position=0,leave=True)
        loss_train_list = []
        if test==10 or test==2:
            continiu

        for Batch in loop_train:
            optimizer.zero_grad()
            
            x = Dataloader.X_numpy[test,Batch*prediction_input_size:(Batch+1)*prediction_input_size]
            y = Dataloader.X_numpy[test,Batch*prediction_input_size: Batch*prediction_input_size+1]

            prediction = model.forward(x)


            loss_train = criterion(prediction, torch.tensor(y.reshape(1,1),dtype=torch.float32).to(device=Utils.Device))
            # Back propagation
            loss_train.backward()
            # Update model parameters
            optimizer.step()
            loss_train_list.append(loss_train)
            
            if Batch%10 == 0:
                # print(loss_train)
                loop_train.set_description(f"{epoch=}- Test : {test}")
                loop_train.set_postfix(
                    loss_batch="{:.7f}".format(torch.tensor(loss_train_list).mean()),refresh=True,)
                
        loss_t.append(torch.tensor(loss_train_list).mean())

162


epoch=1- Test : 28: 100%|██████████| 162/162 [00:02<00:00, 80.21it/s, loss_batch=0.7063621]


162


epoch=2- Test : 9:  24%|██▍       | 39/162 [00:00<00:01, 72.28it/s, loss_batch=0.7019883]

epoch=2- Test : 10:   8%|▊         | 13/162 [00:00<00:02, 58.42it/s, loss_batch=0.6906242]